In [1]:
import json
import numpy as np
import os
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_list():
    json_dir = '/data3/srip_face/diving/code/Label_review/result/'
    f_lst = []
    for itm in os.listdir(json_dir):
        if itm.startswith('seg'):
            f_lst.append(json_dir+itm)
    lst = []
    for itm in f_lst:
        with open(itm, 'r') as f:
            lst.extend(json.load(f))
    random.seed(1234)
    random.shuffle(lst)
    return lst

In [3]:
def save_fg_img(itm, train=True):
    v_dir = '/data3/srip_face/diving/video_first10/'+itm[0]+'.mp4'
    l = min(3, len(itm[1]))
    save_dir = '/data4/srip_face/img/jump_detector/'
    if train:
        save_dir = save_dir + 'train/1fg/'
    else:
        save_dir = save_dir + 'valid/1fg/'

    cap = cv2.VideoCapture(v_dir)
    if cap.isOpened():
        fps = cap.get(5)
        for i in range(l):
            if i==0:
                cap.set(1,fps*itm[1][i]['time'][0]-4)
                for j in range(9):
                    ret, frame = cap.read()
                    if ret:
                        cv2.imwrite(save_dir+itm[0]+'_' + format(i*10+j, '02d') + '.jpg',frame)
            else:
                cap.set(1,fps*itm[1][i]['time'][0])
                for j in range(5):
                    ret, frame = cap.read()
                    if ret:
                        cv2.imwrite(save_dir+itm[0]+'_' + format(i*10+j, '02d') + '.jpg',frame)

In [4]:
def save_bg_img(itm, train=True):
    v_dir = '/data3/srip_face/diving/video_first10/'+itm[0]+'.mp4'
    l = min(3, len(itm[1]))
    save_dir = '/data4/srip_face/img/jump_detector/'
    if train:
        save_dir = save_dir + 'train/0bg/'
    else:
        save_dir = save_dir + 'valid/0bg/'

    cap = cv2.VideoCapture(v_dir)
    if cap.isOpened():
        indx = 0
        fps = cap.get(5)
        n_fr = cap.get(7)
        dur = int(n_fr/fps)-1
        
        cap.set(1, 0.1*fps)
        for j in range(3):
            ret, frame = cap.read()
            if ret:
                cv2.imwrite(save_dir+itm[0]+'_' + format(indx, '02d') + '.jpg',frame)
                indx += 1

        for alpha in [0.35, 0.5, 0.67]:
            midtime = itm[1][0]['time'][1] + alpha*(itm[1][1]['time'][0] - itm[1][0]['time'][1])
            cap.set(1, midtime*fps-1)
            for j in range(3):
                ret, frame = cap.read()
                if ret:
                    cv2.imwrite(save_dir+itm[0]+'_' + format(indx, '02d') + '.jpg',frame)
                    indx += 1

        cap.set(1, fps*(min(itm[1][len(itm)-1]['time'][1]+2, dur)))
        for j in range(3):
            ret, frame = cap.read()
            if ret:
                cv2.imwrite(save_dir+itm[0]+'_' + format(indx, '02d') + '.jpg',frame)
                indx += 1

        for i in range(l):
            for alpha in [0.35, 0.5, 0.67, 0.9]:
                if i ==0 and alpha == 0.35:
                    continue
                midtime = itm[1][i]['time'][0] + alpha*(itm[1][i]['time'][1] - itm[1][i]['time'][0])
                cap.set(1, midtime*fps-1)
                for j in range(3):
                    ret, frame = cap.read()
                    if ret:
                        cv2.imwrite(save_dir+itm[0]+'_' + format(indx, '02d') + '.jpg',frame)
                        indx += 1

In [5]:
lst = get_list()
t = int(0.8*len(lst))
print t

260


In [6]:
print lst[0:1]

[[u'egsanPCCIzQ_000', [{u'cam_view': u'side', u'time': [2.74, 7.400000000000001], u'slow_motion': False}, {u'cam_view': u'side', u'time': [25.166666666666668, 31.040000000000003], u'slow_motion': True}, {u'cam_view': u'front', u'time': [34.379999999999995, 39.885000000000005], u'slow_motion': False}]]]


In [6]:
for idx,itm in enumerate(lst):
    if idx<t:
        save_fg_img(itm, True)
        save_bg_img(itm, True)
    else:
        save_fg_img(itm, False)
        save_bg_img(itm, False)
    print idx

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27